# Network Reconstruction Methods for recovering the degree distribution 

This notebook is implementing some of Raul's methods for recovering the degree distribution of edge-sampled networks. 

## Notation:
* $G(V,E)$ original graph with $N$ nodes, $M$ links. 
* Sampled graph $G'(V',E')$ with $N'$ nodes, $M'$ links.

In [18]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from sampling_utils import *
from plotting_utils import *
import random

from collections import Counter
a4_dims = (11.69,8.27)

from scipy.stats import rv_discrete

## MME Estimator

The first estimator we use is what was described in Ganguly et al's paper as the MME (method of moments estimator), defined as $\hat{k} = k'/p$. It is an unbiased maximum likelihood estimator for the parameter $k$ in a binomial random variable $k' \sim B(k,p)$.

### Notes 
* We round each obtained degree to an integer.
* Because of observed sampled degrees being an integer, all estimated degrees are multiples of $1/p$. E.g. if $p=0.05$ our estimated degrees will all be something like 20, 40 etc.

In [19]:
## Basic estimator for degree sequence which does not redistribute any spare or missing links
def deg_MME_basic(k_seq,sample_prob,as_int=False):
    if sample_prob==0.0:
        return np.zeros(shape=len(k_seq), dtype=float)
    estimated = k_seq/sample_prob
    if (as_int):
        return np.array([*map(int,estimated)])
    return estimated

print(deg_MME_basic(np.array([3,2,5,2]),0.5,True))

[ 6  4 10  4]


## Corrected MME estimator

Here we take into account the estimated number of links $M'/p$. Depending on the value of $p$ (specifically, whether or not $1/p$ is an integer), since each degree is rounded, we may not have the relation $\lfloor 2M'/p \rfloor = \sum_{i=1}^{N'} \lfloor k_i'/p \rfloor $ that is an identity for proper graph degree sequences. We may have too high or too low *total degree* (RHS).

In this procedure, we first calculate the basic MME estimator for the degree sequence and for the number of links, then adjust for differences by either randomly adding links or randomly removing links as required.

### Notes
* If $1/p$ is an integer, this procedure won't change anything as the estimated number of links (multiplied by 2) will be equal to the expected degree sum.

In [20]:
## Estimator which first obtains MME one then deals with rounding discrepency by redistributing links
def deg_MME(kp_seq, probability_keep, redistribute_links=True):

    # Number of nodes/links in sampled graph
    N_prime, M_prime = len(kp_seq), sum(kp_seq)/2

    ## Estimated number of links from sampled graph
    M_estimated = int(M_prime/probability_keep)

    ## Sampled and scaled-up degree distribution
    k_prime = np.array(kp_seq)
    k_est = deg_MME_basic(k_seq=k_prime,sample_prob=probability_keep,as_int=True)

    if (redistribute_links==False):
        return np.array(k_est)

    ## "Left over" stubs from the rounding process
    k_spare = 2*M_estimated - sum(k_est)

    ## Distribute these random stubs if there are any
    if k_spare>0:
        sampled_nodes = random.sample(range(N_prime),k_spare)
        for node in sampled_nodes:
            k_est[node]+=1

    ## If we have given nodes more connections than there are links, randomly remove some
    if k_spare<0:
        non_isolated_nodes = list(filter(lambda ind: k_est[ind]>0, range(N_prime)))
        sampled_nodes = random.sample(non_isolated_nodes,abs(k_spare))
        for node in sampled_nodes:
            k_est[node]-=1
    
    return np.array(k_est) 

## Univariate Risk Minimiser

from Estimation of Vertex Degrees in a Sampled Network, A. Ganguly and E. Kolaczyk. Does not improve upon the MME very much at all.

In [21]:
def deg_RME(kp_seq, probability_keep):
    k_prime = np.array(kp_seq)

    minimiser = lambda k: k**2/(probability_keep * (k + 1 - probability_keep)) + (1-probability_keep)/probability_keep
    return minimiser(k_prime)

# H = nx.erdos_renyi_graph(10,0.1)
# print(deg_RME(H,0.5))
# print(deg_MME(H,0.5,False))

## Monte Carlo estimator

This process starts off with the degree sequence estimated by the MME and tries to improve it by comparing samples from that estimated degree sequence of $G$ with our observed degree sequence in $G'$. 

At each step, a link in the estimated degree distribution is randomly rewired and we calculate the expected degree sequence if we were link-sampling it with probability $p$. If it is closer to the degree sequence of $G'$ than before, then we accept the proposed rewire, otherwise reject.

In [22]:
## Process for randomly redistributing links
def monte_carlo_degree(kp_seq,probability_keep):

    # Number of nodes/links in sampled graph
    N_prime, M_prime = len(kp_seq), sum(kp_seq)/2

    observed_degree = np.array(kp_seq)
    estimated_degree = deg_MME(kp_seq, probability_keep=probability_keep,redistribute_links=True)

    ## Expected degree of sampled network according to binomial
    sampled_sequence = estimated_degree * probability_keep *1.0
    #print(sampled_sequence)

    ## Sum of squared distances as base quality metric
    ssd_current = sum((sampled_sequence - observed_degree)**2)
    #print(observed_degree)
    
    # Commence Monte Carlo process
    # NB stuck in global minima if 1/p is an integer.
    cts_accept=0
    for i in range(15000):
        valid_indices = [ind for ind in range(N_prime) if estimated_degree[ind]>1]

        ## Randomly rewire an edge
        [n1, n2] = random.sample(valid_indices,2)
        # if i==1:
        #     print(n1,n2)

        # ## Ensure we don't leave any isolated nodes
        # while(estimated_degree[n1]<=1):
        #     [n1,n2] = random.sample(range(N_prime),2)
        # estimated_degree[n1]-=1
        # estimated_degree[n2]+=1

        sampled_sequence = estimated_degree * probability_keep
        ssd_new = sum((sampled_sequence - observed_degree)**2)

        ## Reject step if error is larger:
        if (ssd_new > ssd_current):
            estimated_degree[n1]+=1
            estimated_degree[n2]-=1
        else:
            ssd_current = ssd_new
            cts_accept+=1

    return estimated_degree, ssd_current, cts_accept



## Link cascade method
Previously used methods do not take into account isolated nodes, they only compare the difference in degree for nodes that are in $G'$ (i.e. those that end up with degree $>=1$). The following method assumes we know the number of nodes which end up being degree 0 in $G'$.

1. Start off with the MME estimate of the degree sequence (including the zeros for isolated nodes).
2. Rank the nodes in decreasing order of degree.
3. Find the highest rank node that has degree 0, and steal a link from the node directly above it in the rankings.
4. Repeat step 3 until there are no zero-degree nodes.

In [23]:
def degree_cascade(kp_seq, no_isolates, probability_keep):
    
    # Number of nodes/links in sampled graph
    N_prime, M_prime = len(kp_seq), sum(kp_seq)/2

    ## Run monte carlo 
    mme_degrees_wo_isolates = deg_MME(kp_seq=kp_seq,probability_keep=probability_keep,redistribute_links=True)
    mme_degrees = np.concatenate([mme_degrees_wo_isolates,np.zeros(no_isolates,dtype=int)])

    ## get the permutation of degrees into right order and the reverse of this.
    node_index = np.argsort(-1*mme_degrees)
    reverse_index = np.argsort(node_index)

    inds = list(filter(lambda i: mme_degrees[i]==0,node_index))

    ## make sorted list as a copy
    mme_degrees_copy = [mme_degrees[i] for i in node_index]
    while (len(inds)>0):
        # find highest node in the list that has degree 0.
        ind = inds.pop(0)
        mme_degrees_copy[ind-1] -= 1
        mme_degrees_copy[ind] += 1
        if (mme_degrees_copy[ind-1]==0):
            inds.insert(0,ind-1)

    cascaded_degrees = [mme_degrees_copy[i] for i in reverse_index]
    return cascaded_degrees

## Bayes method

This part uses the Bayes method of chapter 6 of my thesis to construct a posterior degree distribution. However, this time, instead of just using Poisson and true prior, we use these 3 estimates of the degree distribution as a prior, named "MME", "Cascaded" and "Monte Carlo".

In [24]:
## Bayes estimate, nb supply the prior as a degree sequence rather than counts of different degrees.
def bayes_approx(prior_as_sequence,sampled_degrees,prob_retain):
    observed = np.array(sampled_degrees)
    N_approx = len(prior_as_sequence)

    ## transform from degree sequence to degree distribution
    deg_counts = Counter(prior_as_sequence)

    ## max value to use for the degree -- pick kmax as the largest support value of the prior.
    k_max = max(deg_counts.elements())

    ## construct prior from approx sequence
    prior = [deg_counts[k]/N_approx for k in range(k_max+1)]
    posterior = np.zeros(N_approx)

    for i in range(N_approx):
        k_observed = observed[i]
        k_range = np.array(range(k_observed,k_max+1))

        ## lambda function so can be applied to numpy array
        binom_ev = lambda k : binom(k,k_observed,prob_retain)

        denom = np.dot(binom_ev(k_range),prior[k_observed:k_max+1])
        numer = np.dot(binom_ev(k_range)*k_range,prior[k_observed:k_max+1])
        if (denom>0):
            posterior[i]=numer/denom
    return posterior
    

In [9]:
## Utility function for mean squared difference between two arrays
def mse(seq1, seq2):
    return sum((seq1-seq2)**2)/len(seq1)

## Main experiment. 

We start with an Erdos-Renyi graph of 1000 nodes and 5000 edges (average degree 10). We make edge samples going from $p=0.05$ to $p=1.0$ going up by 0.05. For each parameter we run 10 experiments and collect the mean and sd to get error bars.

The estimators we are interested in are:
* MME ($k'/p$ estimator)
* "Cascaded" links estimator
* Monte Carlo estimator
* Bayes with MME prior
* Bayes with cascaded links prior
* Bayes with Monte Carlo prior
* Bayes with true prior (we expect this to give an almost exact answer)

The error metric to compare the real and estimated degree sequence is the mean squared error in all cases.

In [123]:
folder = "STACKEX"
net_name = "SX"
p_range = np.linspace(0.1,0.9,9)

G = nx.read_edgelist(folder+"/REAL")

experiments = 10

mme = np.zeros((experiments,len(p_range)))
monte_carlo = np.zeros((experiments,len(p_range)))
cascade = np.zeros((experiments,len(p_range)))
bayes_mc = np.zeros((experiments,len(p_range)))
bayes_csc = np.zeros((experiments,len(p_range)))
bayes_true = np.zeros((experiments,len(p_range)))

def single_run(run_number):
    for ind, p in enumerate(p_range):
        H = nx.read_edgelist(folder+"/"+net_name+str(round(p,1))+"-"+str(run_number))
        kp_seq = [d for (_,d) in nx.degree(H)]

        ## True degree sequence of nodes in G that are in H
        true_degrees = np.array([nx.degree(G)[n] for n in H.nodes])
        kp_full = kp_seq+[0 for _ in range(G.number_of_nodes() - H.number_of_nodes())]
        full_degrees = np.array([nx.degree(G)[n] for n in H.nodes]+[nx.degree(G)[n] for n in G.nodes if n not in H.nodes])

        ## MME and associated error
        # print("Calculating MME")
        #start = time.time()

        mme_seq = deg_MME(kp_seq,p,True)
        mme[run_number,ind] = mse(mme_seq,true_degrees)
        # print(time.time() - start)

        # Cascaded degrees and error
        # print("Cascaded degrees")
        # start = time.time()

        csc_seq = degree_cascade(kp_seq,len(full_degrees) - len(true_degrees),p)
        cascade[run_number,ind] = mse(csc_seq,full_degrees)
        #print(time.time() - start)

        ## Monte carlo improvement
        print("Monte Carlo Degree")
        # start = time.time()

        deg, ssd_error, accepted = monte_carlo_degree(kp_seq,p)
        monte_carlo[run_number,ind] = mse(deg,true_degrees)
        # print(time.time() - start)
        
        # Bayes using monte carlo
        print("Bayes")
        # start = time.time()

        posterior_mc = bayes_approx(deg,kp_seq,p)
        bayes_mc[run_number,ind] = mse(posterior_mc,true_degrees)
        #print(time.time() - start)

        # Bayes using cascaded degrees
        # print("Bayes Cascaded")
        # start = time.time()

        csc_H = np.array(csc_seq[:H.number_of_nodes()])
        csc_bayes_degrees = bayes_approx(csc_H,kp_seq,p)
        bayes_csc[run_number,ind] = mse(csc_bayes_degrees,true_degrees)
        # print(time.time() - start)
        
        # Bayes using true prior
        # print("Bayes True Prior")
        # start = time.time()     

        posterior_true = bayes_approx(true_degrees,kp_seq,p)
        bayes_true[run_number,ind] = mse(posterior_true,true_degrees)
        #print(time.time() - start)

for ex in range(experiments):
    print("Experiment "+str(ex))
    single_run(ex)
    
mme_means, mme_sds = mme.mean(axis=0), mme.std(axis=0)
csc_means, csc_sds = cascade.mean(axis=0), cascade.std(axis=0)
monte_carlo_means, monte_carlo_sds = monte_carlo.mean(axis=0), monte_carlo.std(axis=0)
bayes_mc_means, bayes_mc_sds = bayes_mc.mean(axis=0), bayes_mc.std(axis=0)
bayes_mc_means, bayes_mc_sds = bayes_mc.mean(axis=0), bayes_mc.std(axis=0)
bayes_csc_means, bayes_csc_sds = bayes_csc.mean(axis=0), bayes_csc.std(axis=0)
bayes_true_means, bayes_true_sds = bayes_true.mean(axis=0), bayes_true.std(axis=0)
    

Experiment 0
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Experiment 1
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Experiment 2
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Experiment 3
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Monte Carlo Degree
Bayes
Experiment 4
Monte Carlo Degree
Bayes
Monte Carl

In [124]:
results_df = pd.DataFrame(data=np.transpose([mme_means, mme_sds,csc_means, csc_sds, monte_carlo_means, monte_carlo_sds, bayes_mc_means, bayes_mc_sds,bayes_csc_means, bayes_csc_sds, bayes_true_means, bayes_true_sds]), columns = ["mme_mean", "mme_sd","csc_mean","csc_sd","monte_carlo_mean","monte_carlo_sd","bayes_mc_mean","bayes_mc_sd","bayes_csc_mean","bayes_csc_sd","bayes_true_mean","bayes_true_sd"],index=p_range)
display(results_df)
results_df.to_csv(folder+"/degree_error_dataframe.csv", index=True)

,mme_mean,mme_sd,csc_mean,csc_sd,monte_carlo_mean,monte_carlo_sd,bayes_mc_mean,bayes_mc_sd,bayes_csc_mean,bayes_csc_sd,bayes_true_mean,bayes_true_sd
0.1,333.933686,14.623881,149.785533,6.339386,333.933686,14.623881,334.827368,17.976412,334.823655,17.976096,310.815833,17.173429
0.2,105.629258,4.319701,66.549529,2.213887,105.629258,4.319701,105.177664,4.385885,105.176894,4.385886,99.719351,4.684317
0.3,49.385915,2.820288,34.791987,1.857534,49.404771,2.820185,49.365780,2.827551,49.359822,2.826703,46.827814,2.812289
0.4,29.144715,0.840102,22.999394,0.603890,29.173236,0.868726,28.756119,0.838195,28.751993,0.839632,27.593595,0.887082
0.5,17.776540,0.594388,15.199087,0.477177,17.776540,0.594388,17.896803,0.519504,17.896760,0.519504,17.170802,0.556397
0.6,11.714102,0.456601,12.563617,0.468586,11.709663,0.447574,11.384576,0.439079,11.382669,0.438819,10.874407,0.392665
0.7,7.090026,0.231785,7.401842,0.339521,7.092579,0.237912,6.897615,0.232525,6.899665,0.229161,6.579686,0.267581
0.8,4.238758,0.140996,4.134004,0.119655,4.238542,0.142697,4.052979,0.129510,4.052448,0.128638,3.790642,0.093496
0.9,1.948392,0.058093,2.213038,0.069015,1.955892,0.056969,1.738521,0.067539,1.736631,0.067456,1.591423,0.063267


# Experiments for figure 5 (degree vs estimated degree for small and large p)

In [14]:
folder = "STACKEX"
net_name = "SX"
p_small, p_big = 0.1, 0.9

p=0.9

G = nx.read_edgelist(folder+"/REAL")

H = nx.read_edgelist(folder+"/"+net_name+str(round(p,1))+"-1")
kp_seq = [d for (_,d) in nx.degree(H)]
kp_full = kp_seq+[0 for _ in range(G.number_of_nodes() - H.number_of_nodes())]

## True degree sequence of nodes in G that are in H

true_degrees = np.array([nx.degree(G)[n] for n in H.nodes])
full_degrees = np.array([nx.degree(G)[n] for n in H.nodes]+[nx.degree(G)[n] for n in G.nodes if n not in H.nodes])

mme_seq = deg_MME(kp_full,p,True)

csc_seq = degree_cascade(kp_seq,len(full_degrees) - len(true_degrees),p)

mc_deg, ssd_error, accepted = monte_carlo_degree(kp_full,p)

posterior_mc = bayes_approx(mc_deg,kp_full,p)

csc_H = np.array(csc_seq)
csc_bayes_degrees = bayes_approx(csc_H,kp_full,p)
posterior_true = bayes_approx(full_degrees,kp_full,p)

degree_df = pd.DataFrame(data=np.transpose([full_degrees, mme_seq,csc_seq,mc_deg,posterior_mc,csc_bayes_degrees,posterior_true]), columns=["true_degrees","mme", "cascade","monte_carlo","bayes_mc","bayes_cascade","bayes_true"])
degree_df.to_csv(folder+"/deg-sequences-"+str(round(p,1))+".csv")

# Triangle Count and Distribution Estimators

This section will use the estimators from degree but for the number of triangles per edge and in total.

## Basic functions

First we define some utility functions for counting number of common neighbours between node pairs and triangles per edge, as well as wedge counts

In [14]:
def common_neighbours(n1, n2, G):
    n1neigh = set(G.neighbors(n1))
    n2neigh = set(G.neighbors(n2))
    return len(n1neigh.intersection(n2neigh)), G.has_edge(n1,n2)

def triangles(n1, n2, G):
    if not G.has_edge(n1,n2):
        return 0
    else:
        return len(list(nx.common_neighbors(G,n1,n2)))

def edge_triangle_count(graph,edge_subset=None):
    if edge_subset is not None:
        edges = edge_subset
    else:
        edges = graph.edges()
    tc=[]
    for u,v in edges:
        tc.append(triangles(u,v,graph))
    return tc

def edge_triangle_dict(G):
    tc={}
    for u,v in G.edges():
        tc[(min(u,v),max(u,v))]=triangles(u,v,G)
    return tc


def local_wedge_count(G):
    nodes = sorted(G.nodes())
    ec = []
    for i in range(len(nodes)):
        for j in range(i):
            cn = common_neighbours(nodes[i],nodes[j],G)
            if cn[0]!=0:
                ec.append(cn[0])
    return ec

def wedge_count(G):
    return 0.5*sum([d*(d-1) for n,d in G.degree()])


## MME estimators for triangles and clustering

Some functions for MME estimators of clustering coefficient and triangle count.

In [15]:
def tri_MME(tri_array, p):
    return np.array([round(1.0/p**2*ct) for ct in tri_array])

def scaleup_wedge(H,p):
    return 1.0/p**2*wedge_count(H)

def scaleup_local_wedge(H,p):
    return 1.0/p**2*np.array(H)

def scaleup_clustering_coeff(H,p):
    wc = 1.0/p**2*wedge_count(H)
    t_array = tri_MME(edge_triangle_count(H),p)
    tc = sum(t_array)
    return t_array/wc

def tot_tri_estimate(H,p):
    tri_array = edge_triangle_count(H)/p**3
    return sum(tri_array)

## Probability Mass Functions for triangle count of sampled network

In [16]:
def triangle_likelihood(t,tc,p):
    return comb(t,tc)*np.power((1.0-p*p),t)

def triangle_likelihood_normalised(t,tc,p):
    return comb(t,tc)*np.power(p,2.0*tc)*np.power(1.0-p*p,t-tc)

In [17]:
## for making a cache of pre-calculated probabilities: like a defaultdict but it has 

from collections import defaultdict

class key_dependent_dict(defaultdict):
    def __init__(self,f_of_x):
        super().__init__(None) # base class doesn't get a factory
        self.f_of_x = f_of_x # save f(x)
    def __missing__(self, key): # called when a default needed
        ret = self.f_of_x(key) # calculate default value
        self[key] = ret # and install it in the dict
        return ret

In [30]:
from scipy.stats import poisson

def tri_bayes_approx_pois(tl_sampled, prob_retain, T_lambda):
    m = len(tl_sampled)
    # t_max = len(prior)-1

    posterior = np.zeros(m, dtype=float)

    # ## no need to keep re-evaluating probabilities over and over
    # prob_cache = key_dependent_dict(lambda x : binom(x[0],x[1],prob_retain**2))

    vectorised_binom_num = lambda tc: np.vectorize(lambda t: t*binom(t,tc,prob_retain**2),excluded=["tc"])
    vectorised_binom_den = lambda tc: np.vectorize(lambda t: binom(t,tc,prob_retain**2),excluded=["tc"])

    ## no need to keep re-evaluating probabilities over and over
    prob_cache_num = key_dependent_dict(lambda tc : poisson(T_lambda).expect(vectorised_binom_num(tc), lb=tc))
    prob_cache_den = key_dependent_dict(lambda tc : poisson(T_lambda).expect(vectorised_binom_den(tc), lb=tc))

    for edge in range(m):
        t_observed = tl_sampled[edge]
        # t_range = np.array(range(t_observed, t_max + 1))
        
        # binom_ev = np.vectorize(lambda t: prob_cache[(t,int(t_observed))],otypes=[float])

        # denom = poisson(T_lambda).expect(lambda t: t*binom(t,t_observed,prob_retain**2), lb=t_observed, maxcount=100)
        # numer = poisson(T_lambda).expect(lambda t: binom(t,t_observed,prob_retain**2), lb=t_observed, maxcount=100)
        
        denom = prob_cache_den[t_observed]
        numer = prob_cache_num[t_observed]
        if (denom > 0):
            posterior[edge] = numer/denom
        else:
            posterior[edge] = 0.0

    return np.array(posterior)

def poisson_prior(mean, tol):
    total = 0.0
    ind = 0
    prior_arr = []
    
    while (total < 1.0-tol):
        prior_arr.append(poisson.pmf(ind,mean))
        total+=prior_arr[-1]
        ind+=1
    return np.array(prior_arr)

In [19]:
def tri_bayes_approx_true(tl_sampled, prob_retain, dist):
    m = len(tl_sampled)
    # t_max = len(prior)-1

    posterior = np.zeros(m, dtype=float)

    # ## no need to keep re-evaluating probabilities over and over
    # prob_cache = key_dependent_dict(lambda x : binom(x[0],x[1],prob_retain**2))

    vectorised_binom_num = lambda tc: np.vectorize(lambda t: t*binom(t,tc,prob_retain**2),excluded=["tc"])
    vectorised_binom_den = lambda tc: np.vectorize(lambda t: binom(t,tc,prob_retain**2),excluded=["tc"])

    ## no need to keep re-evaluating probabilities over and over
    prob_cache_num = key_dependent_dict(lambda tc : dist.expect(vectorised_binom_num(tc), lb=tc, maxcount=100))
    prob_cache_den = key_dependent_dict(lambda tc : dist.expect(vectorised_binom_den(tc), lb=tc, maxcount=100))

    for edge in range(m):
        t_observed = tl_sampled[edge]
        # t_range = np.array(range(t_observed, t_max + 1))
        
        # binom_ev = np.vectorize(lambda t: prob_cache[(t,int(t_observed))],otypes=[float])

        # denom = poisson(T_lambda).expect(lambda t: t*binom(t,t_observed,prob_retain**2), lb=t_observed, maxcount=100)
        # numer = poisson(T_lambda).expect(lambda t: binom(t,t_observed,prob_retain**2), lb=t_observed, maxcount=100)
        
        denom = prob_cache_den[t_observed]
        numer = prob_cache_num[t_observed]
        if (denom > 0):
            posterior[edge] = numer/denom
        else:
            posterior[edge] = 0.0

    return np.array(posterior)

def poisson_prior(mean, tol):
    total = 0.0
    ind = 0
    prior_arr = []
    
    while (total < 1.0-tol):
        prior_arr.append(poisson.pmf(ind,mean))
        total+=prior_arr[-1]
        ind+=1
    return np.array(prior_arr)

# Triangle count main experiment

In [21]:
# N,M=1000,10000
# G = nx.gnm_random_graph(N,M)

# G = nx.powerlaw_cluster_graph(10000,5,0.3)

folder="STACKEX"
name="SX"
G = nx.read_edgelist(folder+'/REAL')
M = G.number_of_edges()

Tl_real_full = edge_triangle_count(G)
maxT = max(Tl_real_full)
T_real = sum(Tl_real_full)/3.0

t_counts = Counter(Tl_real_full)
pk = [t_counts[t]/M for t in range(0,maxT + 1)]
true_prior = rv_discrete("true",values= (np.array(range(maxT+1)), pk))

p_range = np.linspace(0.1,0.9,9)
experiments = 10

mme_tri_dist_err = np.zeros((experiments,len(p_range)))
mme_tri_tot_err = np.zeros((experiments,len(p_range)))

bayes_pois_dist_err = np.zeros((experiments,len(p_range)))
bayes_pois_tot_err = np.zeros((experiments,len(p_range)))

bayes_true_dist_err = np.zeros((experiments,len(p_range)))
bayes_true_tot_err = np.zeros((experiments,len(p_range)))

def single_run_tri(run_number):
    for ind, p in enumerate(p_range):
        H = nx.read_edgelist(folder+'/'+name+str(round(p,1))+'-'+str(run_number))

        N_prime, M_prime = H.number_of_nodes(), H.number_of_edges()
        missing_edges = round(M_prime*(1.0/p - 1.0))

        ## Sampled triangle counts and total
        tl_sampled = np.array(edge_triangle_count(H, H.edges())+[0 for _ in range(missing_edges)])
        T_sampled = sum(tl_sampled)/3.0

        ## Real edge triangle count over the subset of H's edges
        tl_real = edge_triangle_count(G,H.edges())

        ## MME scaleup
        mme_tri = tri_MME(tl_sampled,p)
        T_mme = T_sampled/p**3

        mme_tri_dist_err[run_number, ind] = mse(mme_tri[:M_prime],tl_real[:M_prime])
        mme_tri_tot_err[run_number,ind] = (T_mme - T_real)**2

        ## Bayes approx Poisson
        T_lambda = T_mme*3.0/(M_prime + missing_edges)

        bayes_true_tri = tri_bayes_approx_true(tl_sampled[:M_prime], p, true_prior)
        bayes_true_dist_err[run_number,ind] = mse(bayes_true_tri[:M_prime],tl_real[:M_prime])

        # bayes_pois_tri = tri_bayes_approx(tl_sampled[:M_prime],p,poisson_prior(T_lambda,0.000001))
        bayes_pois_tri = tri_bayes_approx_pois(tl_sampled[:M_prime],p,T_lambda)
        bayes_pois_dist_err[run_number,ind] = mse(bayes_pois_tri[:M_prime],tl_real[:M_prime])

        T_bayes_pois = sum(bayes_pois_tri)/(3.0 * p)
        T_bayes_true = sum(bayes_true_tri)/(3.0 * p)
        bayes_pois_tot_err[run_number,ind] = (T_bayes_pois - T_real)**2
        bayes_true_tot_err[run_number,ind] = (T_bayes_true - T_real)**2

        
for ex in range(experiments):
    print("Experiment "+str(ex))
    single_run_tri(ex)

Experiment 0


/opt/homebrew/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:3800: RuntimeWarning: expect(): sum did not converge
  warnings.warn('expect(): sum did not converge', RuntimeWarning)


Experiment 1
Experiment 2
Experiment 3
Experiment 4
Experiment 5
Experiment 6
Experiment 7
Experiment 8
Experiment 9


In [22]:
mme_tri_dist_mean = mme_tri_dist_err.mean(axis=0)
mme_tri_tot_mean = mme_tri_tot_err.mean(axis=0)
mme_tri_dist_sd = mme_tri_dist_err.std(axis=0)
mme_tri_tot_sd = mme_tri_tot_err.std(axis=0)

bayes_tri_dist_mean = bayes_pois_dist_err.mean(axis=0)
bayes_tri_tot_mean = bayes_pois_tot_err.mean(axis=0)
bayes_tri_dist_sd = bayes_pois_dist_err.std(axis=0)
bayes_tri_tot_sd = bayes_pois_tot_err.std(axis=0)

bayes_tri_true_dist_mean = bayes_true_dist_err.mean(axis=0)
bayes_tri_true_tot_mean = bayes_true_tot_err.mean(axis=0)
bayes_tri_true_dist_sd = bayes_true_dist_err.std(axis=0)
bayes_tri_true_tot_sd = bayes_true_tot_err.std(axis=0)


In [23]:
results_df = pd.DataFrame(data=np.transpose([mme_tri_dist_mean, mme_tri_dist_sd,mme_tri_tot_mean, mme_tri_tot_sd, 
bayes_tri_dist_mean, bayes_tri_dist_sd, bayes_tri_tot_mean, bayes_tri_tot_sd, 
bayes_tri_true_dist_mean, bayes_tri_true_dist_sd, bayes_tri_true_tot_mean, bayes_tri_true_tot_sd]), 
columns = ["mme_dist_mean", "mme_dist_sd","mme_tot_mean","mme_tot_sd",
"bayes_dist_mean","bayes_dist_sd","bayes_tot_mean","bayes_tot_sd",
"bayes_dist_true_mean","bayes_dist_true_sd","bayes_tot_true_mean","bayes_tot_true_sd"],index=p_range)
display(results_df)
results_df.to_csv(folder+"/tri_error_dataframe.csv", index=True)

,mme_dist_mean,mme_dist_sd,mme_tot_mean,mme_tot_sd,bayes_dist_mean,bayes_dist_sd,bayes_tot_mean,bayes_tot_sd,bayes_dist_true_mean,bayes_dist_true_sd,bayes_tot_true_mean,bayes_tot_true_sd
0.1,2271.986291,82.261747,2.461447e+09,2.723859e+09,1390.330563,34.044035,2.461447e+09,2.723859e+09,867.319352,16.927738,3.532719e+08,4.231429e+08
0.2,542.639446,13.886696,1.459633e+09,2.056422e+09,1293.814602,24.982014,1.459408e+09,2.055987e+09,386.703641,7.778796,8.216063e+08,1.195872e+09
0.3,226.137372,6.698207,1.632687e+09,1.893586e+09,1171.634853,19.253007,1.631733e+09,1.893795e+09,194.577329,3.742175,1.256592e+09,1.459951e+09
0.4,117.311890,1.421506,3.650151e+08,5.117375e+08,998.064419,8.433404,3.647134e+08,5.115483e+08,108.142787,1.682875,3.144816e+08,4.399140e+08
0.5,67.034513,1.472438,4.477075e+08,5.761282e+08,798.025024,6.792928,4.479194e+08,5.763653e+08,64.461325,1.831170,4.146825e+08,5.416645e+08
0.6,39.945234,1.001169,3.059305e+08,2.663762e+08,585.311539,3.636955,3.059370e+08,2.663427e+08,39.000715,0.603344,2.909429e+08,2.553673e+08
0.7,23.357048,0.539441,1.945954e+08,2.128052e+08,379.979002,5.259731,1.946898e+08,2.146665e+08,22.897836,0.398201,1.889113e+08,2.060667e+08
0.8,12.780708,0.330915,3.804060e+07,3.647559e+07,210.187778,4.745946,4.379946e+07,4.218354e+07,12.623696,0.318116,3.743353e+07,3.584679e+07
0.9,5.375992,0.141840,3.502482e+07,3.246054e+07,104.071357,2.512904,5.414607e+07,5.578499e+07,5.259020,0.151625,3.472433e+07,3.218323e+07


# Real vs estimated triangles

In [36]:
folder="STACKEX"
name="SX"
G = nx.read_edgelist(folder+'/REAL')
M = G.number_of_edges()
p=0.9

# Real triangle counts
Tl_real_full = edge_triangle_count(G)
maxT = max(Tl_real_full)
T_real = sum(Tl_real_full)/3.0

# Turn these real counts into a prior for Bayes
t_counts = Counter(Tl_real_full)
pk = [t_counts[t]/M for t in range(0,maxT + 1)]
true_prior = rv_discrete("true",values= (np.array(range(maxT+1)), pk))

H = nx.read_edgelist(folder+'/'+name+str(round(p,1))+'-1')

N_prime, M_prime = H.number_of_nodes(), H.number_of_edges()
missing_edges = round(M_prime*(1.0/p - 1.0))

## Sampled triangle counts and total
tl_sampled = np.array(edge_triangle_count(H, H.edges())+[0 for _ in range(missing_edges)])
T_sampled = sum(tl_sampled)/3.0

## Real edge triangle count over the subset of H's edges
tl_real = edge_triangle_count(G,H.edges())

## MME scaleup
mme_tri = tri_MME(tl_sampled,p)
T_mme = T_sampled/p**3

## Bayes approx Poisson
T_lambda = T_mme*3.0/(M_prime + missing_edges)
bayes_pois_tri = tri_bayes_approx_pois(tl_sampled[:M_prime],p,T_lambda)

## Bayes true prior
bayes_true_tri = tri_bayes_approx_true(tl_sampled[:M_prime], p, true_prior)

tri_df = pd.DataFrame(data=np.transpose([tl_real[:M_prime], mme_tri[:M_prime], bayes_pois_tri[:M_prime], bayes_true_tri[:M_prime]]), columns=["true","mme","bayes_pois", "bayes_true"])
tri_df.to_csv(folder+"/tri_real_vs_estimated_"+str(round(p,1))+".csv")